## **import du drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Installation de librairies**



In [ ]:
get_ipython().system('pip install PyPDF2')

## **import des modules**

In [ ]:
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr
import re

## **Extraction et préprocessing**



In [ ]:
def extract_text_from_pdf(pdf_path):
    """Extraction avec nettoyage du texte"""
    extracted_text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text = page.extract_text()
                if text:
                    extracted_text += text + "\n"
        return clean_text(extracted_text)
    except Exception as e:
        print(f"Erreur lors de l'extraction: {e}")
        return ""

def clean_text(text):
    """Nettoyage du texte"""
    # Suppression des caractères spéciaux problématiques
    text = re.sub(r'[^\w\s\-.,;:!?()àâäéèêëïîôùûüÿçÀÂÄÉÈÊËÏÎÔÙÛÜŸÇ]', ' ', text)
    # Suppression des espaces multiples
    text = re.sub(r'\s+', ' ', text)
    # Suppression des lignes trop courtes (probablement du bruit)
    lines = [line.strip() for line in text.split('\n') if len(line.strip()) > 20]
    return '\n'.join(lines)


## **Chuncking**

In [ ]:
def smart_chunk_text(text, chunk_size=800, overlap_size=150):
    """
    Chunking  des textes
    """
    # Découpage par phrases
    sentences = re.split(r'(?<=[.!?])\s+', text)

    chunks = []
    current_chunk = ""

    for sentence in sentences:
        # Si ajouter la phrase dépasse la taille, on sauvegarde le chunk
        if len(current_chunk) + len(sentence) > chunk_size and current_chunk:
            chunks.append(current_chunk.strip())
            # Overlap : garder les derniers mots
            words = current_chunk.split()
            overlap_words = words[-overlap_size//5:] if len(words) > overlap_size//5 else words
            current_chunk = ' '.join(overlap_words) + ' ' + sentence
        else:
            current_chunk += ' ' + sentence

    # Ajouter le dernier chunk
    if current_chunk.strip():
        chunks.append(current_chunk.strip())

    return chunks

## **Embedding et Indexation**

In [ ]:
embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

def create_faiss_index(text_chunks):
    """Création d'index FAISS avec normalisation"""
    print(f"Génération des embeddings pour {len(text_chunks)} chunks...")
    embeddings = embedder.encode(
        text_chunks,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=True,
        batch_size=32
    )

    dimension = embeddings.shape[1]
    # Index avec product intérieur (cosine similarity après normalisation)
    index = faiss.IndexFlatIP(dimension)
    index.add(embeddings)

    print(f"✓ Index créé avec {index.ntotal} documents")
    return index, embeddings

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## **recherche sémantique**

In [ ]:
def advanced_search(query, k=5, score_threshold=0.3):
    """
    Recherche avec filtrage par score de similarité
    """
    query_emb = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    scores, doc_ids = index.search(query_emb, k)

    # Filtrer les résultats par score minimum
    results = [
        (text_chunks[i], float(scores[0][j]))
        for j, i in enumerate(doc_ids[0])
        if scores[0][j] > score_threshold
    ]

    return results

## **Modèle génératif**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print(f"✓ Modèle {model_name} chargé")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

✓ Modèle Qwen/Qwen2.5-0.5B-Instruct chargé


## **Fonction RAG**

In [ ]:
def optimized_rag_query(query, top_k=5, max_new_tokens=300):
    """
    Pipeline RAG complet avec prompt engineering amélioré
    """
    # 1. Récupération des chunks pertinents
    retrieved_chunks = advanced_search(query, k=top_k)

    if not retrieved_chunks:
        return "❌ Aucun contexte pertinent trouvé pour répondre à cette question."

    # 2. Construction du contexte avec scores
    context_parts = []
    for i, (text, score) in enumerate(retrieved_chunks[:3], 1):
        context_parts.append(f"[Passage {i}] {text}")

    context = "\n\n".join(context_parts)

    # 3. Prompt engineering optimisé
    prompt = f"""<|im_start|>system
Tu es un assistant utile. Réponds en français basé sur le contexte.<|im_end|>
<|im_start|>user
Contexte: {context}

Question: {query}

Réponds en français.<|im_end|>
<|im_start|>assistant
"""

    # 4. Génération avec paramètres optimisés
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        max_length=1024,  # Plus de contexte
        truncation=True
    )

    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=4,  # Réduction pour plus de vitesse
        temperature=0.7,  # Un peu de créativité
        top_p=0.9,
        early_stopping=True,
        no_repeat_ngram_size=3  # Éviter les répétitions
    )

    # 5. Décodage
    generated_answer = tokenizer.decode(output[0], skip_special_tokens=True)

    # 6. Post-processing
    generated_answer = generated_answer.strip()

    # Ajouter les sources
    sources_info = f"\n\n Sources consultées: {len(retrieved_chunks)} passages (scores: {', '.join([f'{s:.2f}' for _, s in retrieved_chunks[:3]])})"

    return generated_answer + sources_info

## **pipeline**

In [ ]:
pdf_path = '/content/drive/MyDrive/AKAKPO Koffi Moïse/Filière_GIA/Seires_temp/Apprentissage_par_renforcement_profond___2022.pdf'

print(" Extraction du texte...")
extracted_text = extract_text_from_pdf(pdf_path)
print(f"✓ {len(extracted_text)} caractères extraits")

print("\n Création des chunks...")
text_chunks = smart_chunk_text(extracted_text, chunk_size=800, overlap_size=150)
print(f" {len(text_chunks)} chunks créés")

print("\n Génération des embeddings et création de l'index...")
index, embeddings = create_faiss_index(text_chunks)

print("\n✅ Système RAG prêt !")

 Extraction du texte...
✓ 52434 caractères extraits

 Création des chunks...
 106 chunks créés

 Génération des embeddings et création de l'index...
Génération des embeddings pour 106 chunks...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

✓ Index créé avec 106 documents

✅ Système RAG prêt !


## **Test rapide**

In [ ]:
test_queries = [
    "Qu'est-ce que l'apprentissage par renforcement ?",
    "Quelles sont les principales techniques de DRL ?",
    "Comment fonctionne l'algorithme Q-learning ?"
]

print("\n" + "="*70)
print("TESTS DU SYSTÈME")
print("="*70)

for query in test_queries[:1]:  # Tester seulement la première
    print(f"\n Question: {query}")
    answer = optimized_rag_query(query, top_k=5)
    print(f"\n Réponse:\n{answer}")
    print("\n" + "-"*70)



TESTS DU SYSTÈME

 Question: Qu'est-ce que l'apprentissage par renforcement ?

 Réponse:
system
Tu es un assistant utile. Réponds en français basé sur le contexte.
user
Contexte: [Passage 1] Le transfert d apprentissage en RL Le transfert d apprentissage est une technique d apprentissage automatique dans laquelle la connaissance acquise pour une tâche est transférée à une tâche connexe. En RL, le transfert d apprentissage peut être utilisé pour accélérer l apprentissage d une nouvelle tâche en exploitant la connaissance acquise à partir d une tâche connexe. L utilisation de réseaux neuronaux pré-entrainés comme point de départ pour l ap- prentissage d une nouvelle tâche est l une des méthodes les plus prometteuses pour le transfert d apprentissage en RL. Le réseau pré-entrainé peut être affiné pour la nouvelle tâche,permettantàl agentd apprendreplusrapidementetavecmoinsd exemplesd en- traînement.

[Passage 2] est l une des méthodes les plus prometteuses pour le transfert d apprentissa

## **Interface gradio**

In [ ]:
def gradio_rag_interface(question, num_sources):
    """Wrapper pour Gradio avec paramètres configurables"""
    if not question.strip():
        return "⚠️ Veuillez entrer une question."

    return optimized_rag_query(question, top_k=int(num_sources))

# Interface avec plus d'options
iface = gr.Interface(
    fn=gradio_rag_interface,
    inputs=[
        gr.Textbox(
            lines=3,
            label="❓ Votre question sur l'apprentissage par renforcement",
            placeholder="Ex: Qu'est-ce que le Deep Q-Learning ?"
        ),
        gr.Slider(
            minimum=3,
            maximum=10,
            value=5,
            step=1,
            label="📚 Nombre de sources à consulter"
        )
    ],
    outputs=gr.Textbox(
        label="💡 Réponse générée",
        lines=10
    ),
    title="🤖 Système RAG - Deep Reinforcement Learning",
    description="""
    Posez vos questions sur l'apprentissage par renforcement profond.
    Le système récupère les passages les plus pertinents et génère une réponse synthétique.
    """,
    examples=[
        ["Qu'est-ce que l'apprentissage par renforcement ?", 5],
        ["Explique le fonctionnement de l'algorithme DQN", 5],
        ["Quelles sont les différences entre le Q-learning et SARSA ?", 5],
        ["Qu'est-ce que l'exploration vs exploitation ?", 4]
    ],
    theme=gr.themes.Soft()
)

print("\n🌐 Lancement de l'interface Gradio...")
iface.launch(debug=True, share=True)


🌐 Lancement de l'interface Gradio...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://363479256445c45ca5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://003af4277ff08c6f01.gradio.live
Killing tunnel 127.0.0.1:7860 <> https://363479256445c45ca5.gradio.live
